# 持久化

## 快照

In [7]:
import redis
import time
import os

In [3]:
conn = redis.Redis()

In [8]:
def process_logs(conn, path, callback):
    current_file, offset = conn.mget('progress:file', 'progress:position')
    pipe = conn.pipeline()
    def update_progress():
        pipe.mset({
            'progress:file': fname,
            'progress:position': offset
        })
        pipe.excute()
    for fname in sorted(os.listdir(path)):
        if fname < current_file:
            continue
            
            with open(os.path.join(path, fname), mode='rb') as inp:
                if fname == current_file:
                    inp.seek(int(offset, 10))
                else:
                    offset = 0
                current_file = None
                
                for lno, line in enumerate(inp):
                    callback(pipe, line)
                    offset += int(offset)+len(line)
                    if not (lno+1)% 1000:
                        update_progress()
                update_progress()

In [9]:
conn.save()

True

## AOF

|选项|同步频率|
|--|--|
|always|每个Redis写命令都要同步写入硬盘，严重降低Redis速度|
|everysec|每秒执行一次同步，显式地将多个写命令同步到硬盘|
|no|让操作系统来决定应该何时进行同步|

# Redis事务

In [40]:
def list_item(conn, itemid, sellerid, price):
    inventory = 'inventory:%s' % sellerid
    item = '%s.%s' %(itemid, sellerid)
    end = time.time() + 5
    pipe = conn.pipeline()
    while time.time() < end:
        try:
            pipe.watch(inventory)
            if not pipe.sismember(inventory, itemid):
                pipe.unwatch()
                return None
            pipe.multi()
            pipe.zadd('market:', {item:price})
            pipe.srem(inventory, itemid)
            pipe.execute()
            return True
        except redis.exceptions.WatchError:
            pass
    return False

In [57]:
def purchase_item(conn, buyerid, itemid, sellerid, lprice):
    buyer = 'users:%s' %buyerid
    seller = 'users:%s' %sellerid
    item = '%s.%s' %(itemid, sellerid)
    inventory = "inventory:%s" %buyerid
    end = time.time() + 10
    pipe = conn.pipeline()
    
    while time.time() < end:
        try:
            pipe.watch('market:', buyer)
            
            price = pipe.zscore('market:', item)
            funds = float(pipe.hget(buyer, 'funds'))
            if price != lprice or price > funds:
                pipe.unwatch()
                return None
            
            pipe.multi()
            pipe.hincrbyfloat(seller, 'funds', float(price))
            pipe.hincrbyfloat(buyer, 'funds', float(-price))
            pipe.sadd(inventory, itemid)
            pipe.zrem('market:',item)
            pipe.execute()
            return True
        except redis.exceptions.WatchError:
            pass
    return False

In [64]:
conn.hmset('users:1', {'name':'jim','funds':100})
conn.hmset('users:2', {'name':'tom','funds':43})
conn.sadd('inventory:1', 'itemB', 'itemC')
conn.sadd('inventory:2', 'itemD','itemE','itemF')

0

In [65]:
conn.hgetall('users:1')

{b'name': b'jim', b'funds': b'100'}

In [66]:
conn.hgetall('users:2')

{b'name': b'tom', b'funds': b'43'}

In [67]:
conn.smembers('inventory:1')

{b'itemB', b'itemC'}

In [68]:
conn.smembers('inventory:2')

{b'itemA', b'itemD', b'itemE', b'itemF'}

In [69]:
list_item(conn, 'itemB', 1, 40)

True

In [70]:
conn.zrange('market:',0,-1,withscores=True)

[(b'itemB.1', 40.0)]

In [77]:
purchase_item(conn, 2, 'itemB',1, 40.0)

True

In [78]:
conn.zrange('market:',0,-1,withscores=True)

[]

In [79]:
conn.smembers('inventory:2')

{b'itemA', b'itemB', b'itemD', b'itemE', b'itemF'}

In [80]:
conn.smembers('inventory:1')

{b'itemC'}

In [81]:
conn.hgetall('users:1')

{b'name': b'jim', b'funds': b'140'}

In [82]:
conn.hgetall('users:2')

{b'name': b'tom', b'funds': b'3'}

# 性能问题

|性能或者错误|可能原因|解决方法|
|--|--|--|
|单个客户端的性能达到redis-benchmark的50%~60%|不使用流水线的预期性能||
|单个客户端的性能达到redis-benchmark的25%~30%|对于每个命令或者每组命令都创建了新的连接|重用已有的Redis连接|
|客户端返回错误"Cannot assign requested address"(无法分配指定的地址)|对于每个命令或者每组命令都创建了新的连接|重用已有的Redis连接|